In [34]:
!pip install pandas==1.5.3
!pip install sentence-transformers accelerate -U
!pip install transformers[torch] -U

import accelerate
import transformers
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)

Accelerate version: 0.27.2
Transformers version: 4.38.2


In [35]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [36]:
import pandas as pd

epa_data = drive.CreateFile({'id': '1nOdK7YrAzx-2ZpKZCd620rbSG9qLLxZp'})
epa_data.GetContentFile('EPA_EmissionsData.csv')
factor_df = pd.read_csv('EPA_EmissionsData.csv')
print(factor_df.head())

activity_data = drive.CreateFile({'id': '1smUkjfTvvHmx2QiCfDImKOOtdCSb4dTk'})
activity_data.GetContentFile('business_activities_training_data.csv')
activity_df = pd.read_csv('business_activities_training_data.csv')
print(activity_df.head())

# Load the test data
activity_test_data = drive.CreateFile({'id': '1Q6Sm-lxT-cpOJpFeLg_Leyj3w28Ff9GY'})
activity_test_data.GetContentFile('business_activities_test_data.csv')
test_df = pd.read_csv('business_activities_test_data.csv')
print(test_df.head())

   2017 NAICS Code                  2017 NAICS Title       GHG  \
0           111110                   Soybean Farming  All GHGs   
1           111120  Oilseed (except Soybean) Farming  All GHGs   
2           111130          Dry Pea and Bean Farming  All GHGs   
3           111140                     Wheat Farming  All GHGs   
4           111150                      Corn Farming  All GHGs   

                                Unit  \
0  kg CO2e/2021 USD, purchaser price   
1  kg CO2e/2021 USD, purchaser price   
2  kg CO2e/2021 USD, purchaser price   
3  kg CO2e/2021 USD, purchaser price   
4  kg CO2e/2021 USD, purchaser price   

   Supply Chain Emission Factors without Margins  \
0                                          1.223   
1                                          1.223   
2                                          2.874   
3                                          2.874   
4                                          2.874   

   Margins of Supply Chain Emission Factors  \
0 

In [37]:
print(f"\n\nFactor Count: {factor_df.count()}")
print(f"\n\nActivity Count: {activity_df.count()}")



Factor Count: 2017 NAICS Code                                  1016
2017 NAICS Title                                 1016
GHG                                              1016
Unit                                             1016
Supply Chain Emission Factors without Margins    1016
Margins of Supply Chain Emission Factors         1016
Supply Chain Emission Factors with Margins       1016
Reference USEEIO Code                            1016
dtype: int64


Activity Count: Business Activity Description    1374
Vendor                           1374
Cost_USD                         1374
Comment                          1374
2017 NAICS Title                 1374
dtype: int64


In [38]:
# import torch

In [39]:
# device = torch.device('cpu')

In [40]:
label_dict = {value: idx for idx, value in enumerate(activity_df['2017 NAICS Title'].unique())}
activity_df['label'] = activity_df['2017 NAICS Title'].map(label_dict)


In [78]:
import pandas as pd
import numpy as np
import random

# Helper functions
def introduce_minor_errors(text):
    """Introduce minor spelling mistakes in the text."""
    errors_introduced = 0
    max_errors = random.randint(2, 3)  # Decide to introduce 2 or 3 minor errors

    while errors_introduced < max_errors and len(text) > 4:  # Ensure text is long enough to alter
        error_type = random.choice(['substitute', 'omit', 'swap'])
        error_index = random.randint(1, len(text) - 2)  # Avoid beginning and end of the text for simplicity

        if error_type == 'substitute':
            # Substitute a character with a nearby character (mimicking common typing errors)
            substitutions = {'a': 's', 's': 'a', 'd': 'f', 'i': 'o', 'o': 'p', 'e': 'r', 'r': 't'}
            if text[error_index] in substitutions:
                text = text[:error_index] + substitutions[text[error_index]] + text[error_index + 1:]
                errors_introduced += 1

        elif error_type == 'omit':
            # Omit a character
            text = text[:error_index] + text[error_index + 1:]
            errors_introduced += 1

        elif error_type == 'swap':
            # Swap two adjacent characters
            if error_index < len(text) - 1:  # Ensure there's a character to swap with
                text = text[:error_index] + text[error_index + 1] + text[error_index] + text[error_index + 2:]
                errors_introduced += 1
    return text

def introduce_major_errors(text):
    """Replace or scramble parts of the text to introduce major errors."""
    # Randomly choose between scrambling or inserting irrelevant text
    if random.random() < 0.5:
        return ''.join(random.sample(text, len(text)))
    else:
        return "Irrelevant text " + ''.join(random.sample(text, len(text)))
    return text

# Function to randomly apply either minor or major errors to a text
def apply_random_error(text):
    if random.random() < 0.40:  # 15% chance to introduce an error
        #error_type = random.choice(['minor', 'major'])
        #if error_type == 'minor':
        #    return introduce_minor_errors(text)
        #else:
            return introduce_major_errors(text)   # only major errors
    return text


def apply_errors_with_limit(row, fields, max_errors=2):
    """
    Randomly apply errors to a limited number of fields in a row.

    Parameters:
    - row: The DataFrame row to apply errors to.
    - fields: A list of field names to potentially apply errors to.
    - max_errors: Maximum number of fields to apply errors to.
    """
    # Randomly decide how many fields to apply errors to (0 to max_errors)
    errors_to_apply = random.randint(0, max_errors)

    # Randomly select the fields where errors will be applied
    fields_with_errors = random.sample(fields, errors_to_apply)

    # Apply errors to the selected fields
    for field in fields:
        if field in fields_with_errors:
            row[field + ' Error'] = apply_random_error(row[field])
        else:
            row[field + ' Error'] = row[field]

    return row

In [79]:
import numpy as np
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Define the fields to potentially introduce errors
fields = ['Business Activity Description', 'Vendor', 'Comment']

# Apply errors to 2 or fewer fields for each row
activity_df = activity_df.apply(lambda row: apply_errors_with_limit(row, fields), axis=1)

# Combine the possibly altered text fields into a new 'combined_text' column
activity_df['combined_text'] = activity_df['Business Activity Description Error'] + " " + activity_df['Vendor Error'] + " " + activity_df['Comment Error']
# Now, 'combined_text' contains the concatenated texts with either minor or major errors introduced


# Split data into features and labels
# activity_df['combined_text'] = activity_df['Business Activity Description'] + " " + activity_df['Vendor'] + " " + activity_df['Comment']
X = activity_df['combined_text']  # Feature
y = activity_df['label']  # Assuming 'label' is already encoded as numeric labels

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the TF-IDF vectorizer on training data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Initialize and fit the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust parameters as needed
clf.fit(X_train_tfidf, y_train)

# Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Make predictions on the test data
y_pred = clf.predict(X_test_tfidf)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# You can now use clf to make predictions on new data using the same tfidf_vectorizer to transform the new data



Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       0.93      0.96      0.94        26
           2       0.97      0.91      0.94        32
           3       0.90      0.90      0.90        20
           4       0.83      0.93      0.88        27
           5       1.00      0.94      0.97        18
           6       1.00      1.00      1.00        24
           7       1.00      0.89      0.94        27
           8       1.00      1.00      1.00        24
           9       0.97      1.00      0.98        29
          10       0.96      1.00      0.98        23

    accuracy                           0.96       275
   macro avg       0.96      0.96      0.96       275
weighted avg       0.96      0.96      0.96       275

Accuracy: 0.9563636363636364


In [80]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer  # Assuming you've already fitted this with your training data

# Assuming clf is your trained RandomForestClassifier and tfidf_vectorizer is your fitted TF-IDF vectorizer

test_df['encoded_labels'] = test_df['2017 NAICS Title'].apply(lambda x: label_dict.get(x, -1))  # Unseen labels get -1

# Apply errors to 2 or fewer fields for each row
test_df = test_df.apply(lambda row: apply_errors_with_limit(row, fields), axis=1)
# Combine the possibly altered text fields into a new 'combined_text' column
test_df['combined_text'] = test_df['Business Activity Description Error'] + " " + test_df['Vendor Error'] + " " + test_df['Comment Error']

# Prepare the test data features and labels
# test_df['combined_text'] = test_df['Business Activity Description'] + " " + test_df['Vendor'] + " " + test_df['Comment']

X_test_new = test_df['combined_text']  # Feature
y_test_new = test_df['encoded_labels']  # Replace 'label' with the actual column name for labels in your test data

# Transform the test data using the already fitted TF-IDF vectorizer
X_test_new_tfidf = tfidf_vectorizer.transform(X_test_new)

# Make predictions on the new test data
y_pred_new = clf.predict(X_test_new_tfidf)

# Evaluate the model on the new test data
print("New Test Data - Classification Report:\n", classification_report(y_test_new, y_pred_new))
print("New Test Data - Accuracy:", accuracy_score(y_test_new, y_pred_new))


New Test Data - Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96        24
           1       0.94      0.89      0.92        19
           2       1.00      1.00      1.00        20
           3       0.92      0.92      0.92        26
           4       0.93      1.00      0.96        26
           5       0.96      1.00      0.98        23
           6       0.96      0.93      0.95        28
           7       1.00      0.95      0.97        20
           8       0.96      0.89      0.93        28
           9       0.94      0.97      0.96        34
          10       1.00      0.96      0.98        28

    accuracy                           0.96       276
   macro avg       0.96      0.96      0.96       276
weighted avg       0.96      0.96      0.96       276

New Test Data - Accuracy: 0.9565217391304348


In [83]:
# Convert y_pred_new and y_test_new to a DataFrame for easier manipulation
results_df = pd.DataFrame({'combined_text': X_test_new, 'Actual Label': y_test_new, 'Predicted Label': y_pred_new})

# Filter the DataFrame to only include rows where the prediction failed
failed_predictions = results_df[results_df['Actual Label'] != results_df['Predicted Label']]

# Display the details of rows with failed predictions
print("Rows with Failed Predictions:")
pd.set_option('display.max_colwidth', None)  # For pandas versions < 1.0, use -1 instead of None
print(failed_predictions[['combined_text']])
pd.reset_option('display.max_colwidth')

Rows with Failed Predictions:
                                                                                                                                      combined_text
10                                Irrelevant text n ostSfwenroeer yginki arafrim t MinerGuard Education Services Advanced risk management training.
12                                Irrelevant text vilEosntnmmrnsivoe ieg nocrtairne GreenScan Analytics Utilizing advanced monitoring technologies.
47                Irrelevant text talnnau gt Ptusilinrdesimfxblia on FlowTech Plumbing Irrelevant text stae.is  ltqegHro-iieyhrxuc gerauud ttaw ufe
108                                            mn sgsadea  ailanpsltegodnniaTmi TailingsTech Co. veti. lgtnoiovntI yianlunaianfietrei atgmmssasnb a
113                       Irrelevant text oTnrl sspnetrgiircnmdosf eaaroi  d to MineMovers Transport  a fnflotc unofip tiesusrsnrr rmao.roioEettinu
115                 Irrelevant text on rnrdooeasia latitcsc  nrmnyMiotteesmeae Min

In [58]:
# Ensure that '2017 NAICS Title' in both dataframes are of the same format for accurate mapping
# Map 'Supply Chain Emission Factors without Margins' from factor_df to test_df based on '2017 NAICS Title'
test_df['Emission Factor'] = test_df['2017 NAICS Title'].map(
    factor_df.set_index('2017 NAICS Title')['Supply Chain Emission Factors without Margins']
)

# Calculate the emissions for each row before grouping
test_df['Calculated_Emissions'] = test_df['Emission Factor'] * test_df['Cost_USD']

# Selecting unique '2017 NAICS Title' and their corresponding 'Emission Factor' to avoid duplicates
unique_emission_factors = test_df[['2017 NAICS Title', 'Emission Factor']].drop_duplicates()

# Step 1: Match NAICS Titles in test_df with those in factor_df
# This step is simplified due to direct matching by '2017 NAICS Title'.
# In real-world scenarios, consider complexities of matching titles.

# Group 'test_df' by '2017 NAICS Title' and sum the 'Calculated_Emissions' for each group, also count the occurrences
aggregated_emissions = test_df.groupby('2017 NAICS Title').agg(
    Total_Emissions=('Calculated_Emissions', 'sum'),
    Count=('Calculated_Emissions', 'count')
).reset_index()

# Print the total emissions and count for each title
for index, row in aggregated_emissions.iterrows():
    print(f"Title: {row['2017 NAICS Title']}, Total Emissions: {row['Total_Emissions']:.2f}, Count: {row['Count']}")


# Calculate and print overall totals using DataFrame functions
total_count = aggregated_emissions['Count'].sum()
total_emissions = aggregated_emissions['Total_Emissions'].sum()

print(f"\nTotal Count of All Rows: {total_count}")
print(f"Total of Total Emissions: {total_emissions:.2f}")

Title: Automobile Driving Schools, Total Emissions: 1457.40, Count: 28
Title: Bowling Centers, Total Emissions: 5716.62, Count: 34
Title: Historical Sites, Total Emissions: 2015.28, Count: 23
Title: Kidney Dialysis Centers, Total Emissions: 1455.50, Count: 26
Title: New Single-Family Housing Construction (except For-Sale Builders), Total Emissions: 1744.59, Count: 20
Title: Offices of Dentists, Total Emissions: 886.12, Count: 20
Title: Political Organizations, Total Emissions: 1689.65, Count: 28
Title: Sewage Treatment Facilities, Total Emissions: 8468.04, Count: 26
Title: Support Activities for Metal Mining, Total Emissions: 5052.92, Count: 19
Title: Timber Tract Operations, Total Emissions: 3185.31, Count: 24
Title: Uranium-Radium-Vanadium Ore Mining, Total Emissions: 15093.87, Count: 28

Total Count of All Rows: 276
Total of Total Emissions: 46765.30
